# 面向对象编程

面向过程编程、函数式编程和面向对象编程（OOP）是不同的编程范式。  
OOP把一组数据结构和处理它们的方法称为对象（object），把相同行为的对象归为类（class），对象是类的实例。  
OOP通过类的`封装`隐藏内部细节，通过`继承`实现类的特化和泛化，通过`多态`实现基于对象类型的动态分派。  

## 定义类

使用`class`关键字定义类，然后在类中定义方法（method），这样就可以将对象的动态特征描述出来：

In [1]:
'''
类名通常是大写开头的单词，紧接着的(object)，表示该类是从哪个类继承下来的，
没有合适的继承类时就使用object类，这是所有类最终都会继承的类。
'''
class Student(object):
# __init__是一个特殊方法，用于在创建对象时进行初始化操作，可以给对象绑定属性
    def __init__(self, name, age):
        self.name = name
        self.age = age
# 写在类中的函数称之为（对象的）方法，这些方法就是对象可以接收的消息
# PEP 8要求标识符的名字全小写，多个单词用下划线连接，但程序员更倾向于驼峰命名法
    def print_score(self):
        print('%s: %s' % (self.name, self.age))

## 创建和使用对象

In [2]:
def main():
# 创建学生对象的实例并指定其属性
    stu = Student('Marin', 21)
# 给对象发print_score消息
    stu.print_score()
if __name__ == '__main__':
    main()

Marin: 21


## 访问权限问题

上面的代码中，给Student类对象绑定的name和age属性到底具有怎样的访问权限（也称为可见性）？  
在很多面向对象编程语言中，常会将对象的属性设置为私有的（private）或受保护的（protected），  
简单地说就是不允许外界访问，而对象的方法通常是公开的（public）。  
Python中访问权限只有公开和私有两种，若希望属性或方法私有，可以在命名时用两个下划线开头。

In [ ]:
class Test:
    def __init__(self, name):
        self.__name = name
    def __print(self):
        print(self.__name)
def main():
    test = Test('Marin')
# AttributeError: 'Test' object has no attribute '__name'
    test.__print()
# AttributeError: 'Test' object has no attribute '__print'
    print(test.__name)
if __name__ == "__main__":
    main()

Python没有从语法上严格保证私有属性或方法的私密性，只进行了换名以妨碍访问，  
若知道换名规则仍然可以访问它们，之所以这样设定，是因为“开放比封闭要好”。

In [3]:
class Test:
    def __init__(self, name):
        self.__name = name
    def __print(self):
        print(self.__name)
def main():
    test = Test('Marin')
    test._Test__print()
    print(test._Test__name)
if __name__ == "__main__":
    main()

Marin
Marin


实际开发中不建议将属性设置为私有的，因为这会导致子类无法访问。  
多数程序员会让属性名以单下划线开头来表示该属性是受保护的，  
这种做法并不基于语法规则，只是隐喻本类之外的代码在访问该属性时要保持慎重。

这里简单总结一下封装，个人对封装的理解是，隐藏一切可隐藏的实现细节，只向外界提供简单的编程接口。  
在类中定义的方法其实就是把数据和对数据的操作封装起来，创建对象后只需调用方法就能执行方法中的代码，  
因此只要知道方法名和传入的参数（方法的外部视图），而不需要知道方法内部的实现细节（方法的内部视图）。

## @property装饰器

上文建议过将属性命名以单下划线开头，通过这种方式来暗示属性是受保护的，不建议外界直接访问，  
可以通过`@property`装饰器访问保护属性，@property装饰器修饰的方法相当于属性的`getter（）`。  
如果想修改属性的值或者删除指定属性，则要用到`setter`装饰器和`deleter`装饰器。

In [4]:
class Person(object):
    def __init__(self, name, age):
        self._name = name
        self._age = age
# @property装饰器
    @property
    def name(self):
        return self._name
    @property
    def age(self):
        return self._age
# setter装饰器
    @name.setter
    def name(self, name):
        self._name = name
    @age.setter
    def age(self, age):
        self._age = age
# 这里写一个play（）
    def play(self):
        if self._age <= 18:
            print('%s正在下棋' % self._name)
        else:
            print('%s正在打牌' % self._name)
def main():
    person = Person('Marin', 14)
    person.play()
    person.name = 'Arthur'
    person.age = 22
    person.play()
if __name__ == '__main__':
    main()

Marin正在下棋
Arthur正在打牌


## \_\_slots\_\_魔法

Python是动态语言，允许在程序运行时给对象绑定新的属性或方法，也可以对已绑定的属性和方法解绑。  
如果要限定自定义类型的对象只能绑定某些属性，可以在类中定义`__slots__`变量。  
需要注意__slots__的限定作用只对当前类的对象生效，对子类不起作用。

In [ ]:
class Person(object):
# 限定Person类对象只能绑定_name, _age和_gender属性
    __slots__ = ('_name', '_age', '_gender')
    def __init__(self, name, age):
        self._name = name
        self._age = age
def main():
    person = Person('Marin', 14)
    person._gender = 'Male'
# AttributeError: 'Person' object has no attribute '_shape'
    person._shape = 'overweight'
if __name__ == '__main__':
    main()

## 静态方法和类方法

写在类中的方法并不需要都是实例方法，例如定义一个Triangle类，通过传入三条边长来构造三角形，  
并提供计算周长和面积的方法。由于传入的三条边长未必能构成三角形，可以先写一个方法验证。  
这个方法显然不是实例方法，因为在调用它时对象尚未创建，可以用静态方法来解决这类问题。  
实例方法通过给对象发消息调用，而静态方法和类方法都是通过给类发消息来调用的。

In [5]:
from math import sqrt
class Triangle(object):
    def __init__(self, a, b, c):
        self._a = a
        self._b = b
        self._c = c
# 静态方法要用修饰器@staticmethod来标识
    @staticmethod
    def is_valid(a, b, c):
        return a + b > c and b + c > a and a + c > b
# 实例方法
    def perimeter(self):
        return self._a + self._b + self._c
    def area(self):
        half = self.perimeter() / 2
        return sqrt(half * (half - self._a) *
                    (half - self._b) * (half - self._c))
def main():
    a, b, c = 3, 4, 5
    if Triangle.is_valid(a, b, c):
        t = Triangle(a, b, c)
        print(t.perimeter())
        print(t.area())
# 事实上，同样可以通过给类发消息来调用实例方法，但是要求传入接收消息的对象作为参数
# print(Triangle.perimeter(t))
# print(Triangle.area(t))
    else:
        print('无法构成三角形')
if __name__ == '__main__':
    main()

12
6.0


实例方法一般用`self`作为第一个参数，类方法中默认使用的第一个参数则是`cls`，cls代表类本身  
（类本身也是对象，又称类的元数据对象），可以用cls来实例化一个对象，而self则是类的一个实例对象。 

In [6]:
from time import time, localtime, sleep
class Clock(object):
    def __init__(self, hour=0, minute=0, second=0):
        self._hour = hour
        self._minute = minute
        self._second = second
# 类方法要用修饰器@classmethod来标识
    @classmethod
    def now(cls):
        ctime = localtime(time())
        return cls(ctime.tm_hour, ctime.tm_min, ctime.tm_sec)
# 实例方法
    def show(self):
        return '%02d:%02d:%02d' % \
               (self._hour, self._minute, self._second)
def main():
# 通过类方法创建对象并获取系统时间
    clock = Clock.now()
    while True:
        print(clock.show())
        break
if __name__ == '__main__':
    main()

20:17:56


## 类之间的关系

第一种`is-a`，称为继承，比如手机和电子产品的关系。第二种`has-a`，称为关联，比如汽车和引擎的关系。  
第三种`use-a`，称为依赖，司机有一个驾驶的行为（方法），方法的参数用到了汽车，那么司机和汽车的关系就是依赖。  
可以使用`UML`（统一建模语言）进行面向对象建模，其中就包括用标准图形符号描述类之间的关系，此处不再赘述。

## 继承与多态

可以在已有类的基础上创建新类，其中一种做法是让一个类从另一个类那里直接继承属性和方法。  
提供继承信息的称为父类，也叫超类或基类。得到继承信息的称为子类，也叫派生类或衍生类。  
子类除了继承父类提供的属性和方法，还可以定义自己特有的属性和方法。  
在实际开发中，常会用子类对象去替换掉一个父类对象，对应的原则称之为里氏替换原则。

In [7]:
class Person(object):
    def __init__(self, name, age):
        self._name = name
        self._age = age
    @property
    def name(self):
        return self._name
    @property
    def age(self):
        return self._age
    @name.setter
    def name(self, name):
        self._name = name
    @age.setter
    def age(self, age):
        self._age = age
    def play(self):
        if self._age <= 18:
            print('%s正在下棋' % self._name)
        else:
            print('%s正在打牌' % self._name)
class Student(Person):
    def __init__(self, name, age, grade):
        super().__init__(name, age)
        self._grade = grade
    @property
    def grade(self):
        return self._grade
    @grade.setter
    def grade(self, grade):
        self._grade = grade
    def study(self, course):
        print('%s的%s正在学习%s' % (self._grade, self._name, course))
class Teacher(Person):
    def __init__(self, name, age, title):
        super().__init__(name, age)
        self._title = title
    @property
    def title(self):
        return self._title
    @title.setter
    def title(self, title):
        self._title = title
    def teach(self, course):
        print('%s%s正在讲授%s' % (self._name, self._title, course))
def main():
    stu = Student('Jack', 20, '二年级')
    stu.play()
    stu.study('数学')
    t = Teacher('Professor ',42, 'Alex')
    t.play()
    t.teach('Python程序设计')
if __name__ == '__main__':
    main()

Jack正在打牌
二年级的Jack正在学习数学
Professor 正在打牌
Professor Alex正在讲授Python程序设计


子类在继承了父类的方法后，可以对父类已有的方法给出新的实现版本，这个动作叫重写（override）。  
通过方法重写可以让父类的同一个行为在子类中拥有不同的实现版本，  
在调用这个经过子类重写的方法时，不同的子类对象会表现出不同的行为，也就是所谓的多态。

In [8]:
from abc import ABCMeta, abstractmethod
class Pet(object, metaclass=ABCMeta):
    def __init__(self, nickname):
        self._nickname = nickname
    @abstractmethod
    def make_voice(self):
        pass
class Dog(Pet):
    def make_voice(self):
        print('%s: 汪汪' % self._nickname)
class Cat(Pet):
    def make_voice(self):
        print('%s: 喵喵' % self._nickname)
def main():
    pets = [ Cat('汤姆'), Dog('大黄')]
    for pet in pets:
        pet.make_voice()
if __name__ == '__main__':
    main()

汤姆: 喵喵
大黄: 汪汪


上面的代码将Pet类处理成了一个抽象类，所谓抽象类就是不能够创建对象的类，其存在就是为了让其他类去继承它。  
Python并未提供对抽象类的支持，可以通过`abc`模块的`ABCMeta`元类和`abstractmethod`包装器达到抽象类的效果。  
如果一个类中存在抽象方法，这个类就不能够实例化（创建对象）。  
上文中Dog和Cat两个子类分别对Pet类中的`make_voice（）`抽象方法进行了重写，  
因此在main（）中调用该方法时，就表现出了多态（同样的方法做了不同的事情）。